# Notebook to generate code for functions.py and indicators.py

**Warning** developer use only!

In [3]:
import inspect
import project

import pandas as pd

from mintalib import core
from mintalib.utils import sample_prices, random_walk

from pprint import pformat


# TODO move core_functions, core_indicators to utils ?

In [4]:

def core_functions(first_param=('series', 'prices')):
    """ list of core functions """

    result = []
    for k, v in vars(core).items():
        if isinstance(v, type) or not callable(v):
            continue

        if v.__name__.startswith("_"):
            continue

        params = list(inspect.signature(v).parameters)

        if params[0] not in first_param:
            continue

        result.append(k)

    return result


print(core_functions())

['calc_avgprice', 'calc_typprice', 'calc_wclprice', 'calc_midprice', 'calc_log', 'calc_exp', 'calc_roc', 'calc_diff', 'calc_min', 'calc_max', 'calc_sum', 'calc_mad', 'calc_stdev', 'calc_sma', 'calc_ema', 'calc_rma', 'calc_wma', 'calc_dema', 'calc_tema', 'calc_ma', 'calc_rsi', 'calc_plusdi', 'calc_minusdi', 'calc_adx', 'calc_trange', 'calc_atr', 'calc_natr', 'calc_latr', 'calc_psar', 'calc_cci', 'calc_cmf', 'calc_mfi', 'calc_bop', 'calc_bbands', 'calc_keltner', 'calc_macd', 'calc_ppo', 'calc_slope', 'calc_curve', 'calc_stoch', 'calc_streak']


In [5]:

def get_series(data, item=None, *, default_item='close'):
    if hasattr(data, 'columns'):
        if item is None:
            item = default_item

        return data[item]

    if item is not None:
        raise ValueError("Cannot specify item with %s input!", type(data).__name__)

    return data


print("***")
print(inspect.getsource(get_series))
print("***")



***
def get_series(data, item=None, *, default_item='close'):
    if hasattr(data, 'columns'):
        if item is None:
            item = default_item

        return data[item]

    if item is not None:
        raise ValueError("Cannot specify item with %s input!", type(data).__name__)

    return data

***


In [6]:
prices = sample_prices()
get_series(prices)


date
2021-08-24    98.76
2021-08-25    99.09
2021-08-26    97.32
2021-08-27    97.14
2021-08-30    97.01
              ...  
2022-08-16    88.10
2022-08-17    87.09
2022-08-18    86.76
2022-08-19    86.63
2022-08-22    87.11
Name: close, Length: 260, dtype: float64

In [7]:
series = random_walk()
get_series(series)


def make_function(func, accessor=None, verbose=False):
    fname = func.__qualname__
    module = func.__module__.rpartition('.')[2]
    qname = f"{module}.{fname}"
    indent = "    " if accessor else ""


    signature = inspect.signature(func)
    parameters = list(signature.parameters.values())
    Parameter = inspect.Parameter

    if verbose:
        print("signature", signature)
        print("parameters", parameters)

    newparams = []
    extras = dict()
    ftype = parameters[0].name

    for i, p in enumerate(parameters):
        if i == 0 and accessor:
            if p.name in ('series', 'prices'):
                p = p.replace(name='self')
            else:
                raise ValueError("Expected series or prices as first parameter")

        if p.name == 'wrap':
            extras.update(wrap=True)
            continue

        if p.name in ['period']:
            p = p.replace(annotation=int)

        for typ in (int, float, bool):
            if isinstance(p.default, typ):
                p = p.replace(annotation=typ)

        newparams.append(p)

    if ftype == 'series' and accessor != 'series':
        default = 'close' if accessor else None
        p = Parameter('item', Parameter.KEYWORD_ONLY, default=default, annotation=str)
        newparams.append(p)


    if verbose:
        print("newparams", newparams)
        print("extras", extras)

    newsig = inspect.Signature(newparams)

    name = func.__name__.partition('_')[2]

    def argument(arg):
        if arg == 'self':
            return ftype
        if arg in ('series', 'prices', 'period'):
            return arg
        return f"{arg}={arg}"

    args = [argument(p.name) for p in newparams if p.name not in ['item']]
    args += [f"{k}={v!r}" for k, v in extras.items()]

    args = ", ".join(args)

    code = ""


    if name.isupper():
        code += indent + "# noinspection PyPep8Naming\n"

    wrapper = "utils.wrap_accessor" if accessor else "utils.wrap_function"

    code += indent + f"@{wrapper}({qname})\n"
    code += indent + f"def {name}" + str(newsig) + ":\n"

    if ftype == 'series':
        if accessor == 'prices':
            code += indent + "    series = self.prices[item]\n"
        elif accessor == 'series':
            code += indent + "    series = self.series\n"
        else:
            code += indent + "    series = get_series(series, item=item)\n"

    if ftype == 'prices':
        if accessor == 'prices':
            code += indent + "    prices = self.prices\n"


    code += indent + f"    return {qname}({args})\n"

    return code


print(make_function(core.calc_ema))
print(make_function(core.calc_trange))
print(make_function(core.calc_ema, accessor='series'))
print(make_function(core.calc_ema, accessor='prices'))
print(make_function(core.calc_trange, accessor='prices'))




@utils.wrap_function(core.calc_ema)
def ema(series, period: int, *, mixed: bool = True, item: str = None):
    series = get_series(series, item=item)
    return core.calc_ema(series, period, mixed=mixed, wrap=True)

@utils.wrap_function(core.calc_trange)
def trange(prices, *, log_prices: bool = False, percent: bool = False):
    return core.calc_trange(prices, log_prices=log_prices, percent=percent, wrap=True)

    @utils.wrap_accessor(core.calc_ema)
    def ema(self, period: int, *, mixed: bool = True):
        series = self.series
        return core.calc_ema(series, period, mixed=mixed, wrap=True)

    @utils.wrap_accessor(core.calc_ema)
    def ema(self, period: int, *, mixed: bool = True, item: str = 'close'):
        series = self.prices[item]
        return core.calc_ema(series, period, mixed=mixed, wrap=True)

    @utils.wrap_accessor(core.calc_trange)
    def trange(self, *, log_prices: bool = False, percent: bool = False):
        prices = self.prices
        return core.calc

In [8]:
def make_functions(relative=False):
    buffer = "# Do not edit! File generated automatically...\n\n"

    package = '.' if relative else 'mintalib'

    buffer += f"from {package} import core\n"
    buffer += f"from {package} import utils\n"
    buffer += "\n\n"

    buffer += inspect.getsource(get_series) + "\n\n"
    names = core_functions()

    for name in names:
        func = getattr(core, name)
        text = make_function(func)
        buffer += text + "\n\n"

    return buffer

code = make_functions()
print(code)

exec(code, {})



# Do not edit! File generated automatically...

from mintalib import core
from mintalib import utils


def get_series(data, item=None, *, default_item='close'):
    if hasattr(data, 'columns'):
        if item is None:
            item = default_item

        return data[item]

    if item is not None:
        raise ValueError("Cannot specify item with %s input!", type(data).__name__)

    return data


@utils.wrap_function(core.calc_avgprice)
def avgprice(prices):
    return core.calc_avgprice(prices)


@utils.wrap_function(core.calc_typprice)
def typprice(prices):
    return core.calc_typprice(prices)


@utils.wrap_function(core.calc_wclprice)
def wclprice(prices):
    return core.calc_wclprice(prices)


@utils.wrap_function(core.calc_midprice)
def midprice(prices):
    return core.calc_midprice(prices)


@utils.wrap_function(core.calc_log)
def log(series, *, item: str = None):
    series = get_series(series, item=item)
    return core.calc_log(series)


@utils.wrap_function(core.cal

In [9]:
code = make_functions()
exec(code, {})

code = make_functions(relative=True)

functions = project.pkgdir / "functions.py"

print(f"Updating {functions.name} ...")

functions.write_text(code)


Updating functions.py ...


7028

In [12]:
def core_indicators():
    """ list of core indicator """

    result = []
    for k, v in vars(core).items():
        if not isinstance(v, type):
            continue

        if not v.__name__.isupper():
            continue

        result.append(k)

    return result


print(core_indicators())

['PRICE', 'AVGPRICE', 'TYPPRICE', 'WCLPRICE', 'MIDPRICE', 'VOLUME', 'LOG', 'EXP', 'ROC', 'DIFF', 'MIN', 'MAX', 'SUM', 'MAD', 'STDEV', 'SMA', 'EMA', 'RMA', 'WMA', 'DEMA', 'TEMA', 'RSI', 'ADX', 'PLUSDI', 'MINUSDI', 'TRANGE', 'ATR', 'NATR', 'LATR', 'PSAR', 'CCI', 'CMF', 'MFI', 'BOP', 'BBANDS', 'KELTNER', 'MACD', 'PPO', 'SLOPE', 'CURVE', 'STOCH', 'EVAL']


In [13]:

def make_indicators(relative=False):
    prefix = '' if relative else 'mintalib'

    buffer = "# Do not edit! file generated automatically. see make-indicators.py\n\n"
    buffer += "''' Mintalib indicators library '''\n\n"
    names = [n for n in dir(core) if n.isupper() and n.isalpha()]
    for name in core_indicators():
        buffer += f"from {prefix}.core import {name}\n"

    names = pformat(names, compact=True, indent=0)
    buffer += f"\n__all__ = {names}\n"

    return buffer

code = make_indicators()
print(code)

exec(code, {})


# Do not edit! file generated automatically. see make-indicators.py

''' Mintalib indicators library '''

from mintalib.core import PRICE
from mintalib.core import AVGPRICE
from mintalib.core import TYPPRICE
from mintalib.core import WCLPRICE
from mintalib.core import MIDPRICE
from mintalib.core import VOLUME
from mintalib.core import LOG
from mintalib.core import EXP
from mintalib.core import ROC
from mintalib.core import DIFF
from mintalib.core import MIN
from mintalib.core import MAX
from mintalib.core import SUM
from mintalib.core import MAD
from mintalib.core import STDEV
from mintalib.core import SMA
from mintalib.core import EMA
from mintalib.core import RMA
from mintalib.core import WMA
from mintalib.core import DEMA
from mintalib.core import TEMA
from mintalib.core import RSI
from mintalib.core import ADX
from mintalib.core import PLUSDI
from mintalib.core import MINUSDI
from mintalib.core import TRANGE
from mintalib.core import ATR
from mintalib.core import NATR
from mintalib

In [14]:
code = make_indicators()
exec(code, {})

code = make_indicators(relative=True)

indicators = project.pkgdir / "indicators.py"

print(f"Updating {indicators.name} ...")

indicators.write_text(code)


Updating indicators.py ...


1452

In [1]:
REGISTER_ACCESSORS = '''
def register_accessors(name='ta', *, force=False):
    """ Register Pandas Accessors """

    if force or not hasattr(pd.Series, name):
        pd.api.extensions.register_series_accessor(name)(SeriesAccessor)

    if force or not hasattr(pd.DataFrame, name):
        pd.api.extensions.register_dataframe_accessor(name)(PricesAccessor)
'''


SERIES_ACCESSOR = '''
class SeriesAccessor:
    """ Pandas Series Accessor """

    def __init__(self, series):
        assert isinstance(series, pd.Series)
        self.series = series
'''

PRICES_ACCESSOR = '''
class PricesAccessor:
    """ Pandas Prices Accessor """

    def __init__(self, prices):
        assert isinstance(prices, pd.DataFrame)
        self.prices = prices
'''

def make_accessors(relative=False):
    buffer = "# Do not edit! File generated automatically...\n\n"

    package = '.' if relative else 'mintalib'

    buffer += f"import pandas as pd\n\n"
    buffer += f"from {package} import core\n"
    buffer += f"from {package} import utils\n"
    buffer += "\n"

    buffer += REGISTER_ACCESSORS + "\n"

    buffer += SERIES_ACCESSOR + "\n"

    accessor = 'series'
    names = core_functions(['series'])

    for name in names:
        func = getattr(core, name)
        text = make_function(func, accessor=accessor)
        buffer += text + "\n"

    buffer += PRICES_ACCESSOR + "\n"

    accessor = 'prices'
    names = core_functions(['series', 'prices'])

    for name in names:
        func = getattr(core, name)
        text = make_function(func, accessor=accessor)
        buffer += text + "\n"

    return buffer

code = make_accessors()
print(code)
exec(code, {})



NameError: name 'core_functions' is not defined

In [ ]:

code = make_accessors()
exec(code, {})

code = make_accessors(relative=True)

#accessors = project.pkgdir / "accessors.py"
#print(f"Updating {accessors.name} ...")
#accessors.write_text(code)
